<a href="https://colab.research.google.com/github/MatheusHaliski/Android/blob/main/Aula02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importar bibliotecas
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [3]:
#criar contexto
sc= SparkSession.builder.master('local[*]').appName('basics').getOrCreate()

url = 'https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/realstate.csv'

#add files
sc.sparkContext.addFile(url)

# criar dataframe
df = sc.read.csv(SparkFiles.get("realstate.csv"), header=True, inferSchema=True)

In [6]:
#Demonstrar base
df.head()

Row(F1=0, id=2595, neighbourhood='Midtown', latitude=40.75362, longitude=-73.98377, room_type='Entire home/apt', price=225, days_occupied_in_2019=15, minimum_nights=10, number_of_reviews=48, reviews_per_month=0.39, availability_2020=1)

In [7]:
#mostrar o schema
df.printSchema

<bound method DataFrame.printSchema of DataFrame[F1: int, id: int, neighbourhood: string, latitude: double, longitude: double, room_type: string, price: int, days_occupied_in_2019: int, minimum_nights: int, number_of_reviews: int, reviews_per_month: double, availability_2020: int]>

In [8]:
# para usar o SQL tradicional, precisamos criar a view
df.createOrReplaceTempView('realstate')


In [9]:
#1. selecionar os bairros de forma geral
#a. SQL query
df.select('neighbourhood').show()


+------------------+
|     neighbourhood|
+------------------+
|           Midtown|
|          Brooklyn|
|         Manhattan|
|Bedford-Stuyvesant|
|         Manhattan|
|   Lower East Side|
|         Manhattan|
|        Park Slope|
|         Manhattan|
|      Williamsburg|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|        Park Slope|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|         Manhattan|
|      East Village|
+------------------+
only showing top 20 rows



In [10]:
#b. Utilizar o dataframe
df.select('neighbourhood').show()


+------------------+
|     neighbourhood|
+------------------+
|           Midtown|
|          Brooklyn|
|         Manhattan|
|Bedford-Stuyvesant|
|         Manhattan|
|   Lower East Side|
|         Manhattan|
|        Park Slope|
|         Manhattan|
|      Williamsburg|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|        Park Slope|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|          Brooklyn|
|         Manhattan|
|      East Village|
+------------------+
only showing top 20 rows



In [11]:
#c. utilizando funções do spark.sql
import pyspark.sql.functions as f


In [14]:
#2. seleção de bairros e preços com condição, price >120
#a. sql
df.select('neighbourhood', 'price').filter(df.price > 120).show()
query = 'Select neighbourhood, price from realstate where price > 120'
sc.sql(query).show()

+----------------+-----+
|   neighbourhood|price|
+----------------+-----+
|         Midtown|  225|
|       Manhattan|  200|
| Lower East Side|  150|
|    Williamsburg|  140|
|        Brooklyn|  199|
|        Brooklyn|  299|
|    East Village|  180|
|       Manhattan|  135|
|       Manhattan|  150|
|        Flatbush|  150|
|Long Island City|  130|
| Upper West Side|  151|
|    Williamsburg|  228|
|      Greenpoint|  149|
|        Kips Bay|  200|
|    Williamsburg|  150|
|       Manhattan|  180|
|        Brooklyn|  200|
|        Brooklyn|  250|
|     East Harlem|  225|
+----------------+-----+
only showing top 20 rows

+----------------+-----+
|   neighbourhood|price|
+----------------+-----+
|         Midtown|  225|
|       Manhattan|  200|
| Lower East Side|  150|
|    Williamsburg|  140|
|        Brooklyn|  199|
|        Brooklyn|  299|
|    East Village|  180|
|       Manhattan|  135|
|       Manhattan|  150|
|        Flatbush|  150|
|Long Island City|  130|
| Upper West Side|  151|

In [16]:
#b. utilizando o Spark SQL e dataframe
df.select('neighbourhood', 'price').filter(df.price > 120).show()
df.where(df.price > 120).groupBy('neighbourhood').count().withColumnRenamed('count', 'contagem').orderBy('contagem', ascending=False).show(5)


+----------------+-----+
|   neighbourhood|price|
+----------------+-----+
|         Midtown|  225|
|       Manhattan|  200|
| Lower East Side|  150|
|    Williamsburg|  140|
|        Brooklyn|  199|
|        Brooklyn|  299|
|    East Village|  180|
|       Manhattan|  135|
|       Manhattan|  150|
|        Flatbush|  150|
|Long Island City|  130|
| Upper West Side|  151|
|    Williamsburg|  228|
|      Greenpoint|  149|
|        Kips Bay|  200|
|    Williamsburg|  150|
|       Manhattan|  180|
|        Brooklyn|  200|
|        Brooklyn|  250|
|     East Harlem|  225|
+----------------+-----+
only showing top 20 rows

+------------------+--------+
|     neighbourhood|contagem|
+------------------+--------+
|         Manhattan|    1956|
|          Brooklyn|    1488|
|           Midtown|     319|
|      Williamsburg|     300|
|Bedford-Stuyvesant|     279|
+------------------+--------+
only showing top 5 rows



In [21]:
#c. usando funções
import pyspark.sql.functions as func
df.select('price','neighbourhood').where(func.col('neighbourhood')=='The Bronx').groupby('neighbourhood').max().show(5)


+-------------+----------+
|neighbourhood|max(price)|
+-------------+----------+
|    The Bronx|       970|
+-------------+----------+



In [24]:
#3.Contagem de apartamentos por bairro, com preços maiores que 120
#a. sql puro
query = 'Select neighbourhood, count(*) as contagem from realstate where price > 120 group by neighbourhood order by contagem desc'
sc.sql(query).show(5)

+------------------+--------+
|     neighbourhood|contagem|
+------------------+--------+
|         Manhattan|    1956|
|          Brooklyn|    1488|
|           Midtown|     319|
|      Williamsburg|     300|
|Bedford-Stuyvesant|     279|
+------------------+--------+
only showing top 5 rows



In [23]:
#b. Contagem de apartamentos por bairro, com preços maiores que 120 utilizando Spark SQL/dataframe
df.select('neighbourhood', 'price').filter(df.price > 120).groupBy('neighbourhood').count().withColumnRenamed('count', 'contagem').orderBy('contagem', ascending=False).show(5)



+------------------+--------+
|     neighbourhood|contagem|
+------------------+--------+
|         Manhattan|    1956|
|          Brooklyn|    1488|
|           Midtown|     319|
|      Williamsburg|     300|
|Bedford-Stuyvesant|     279|
+------------------+--------+
only showing top 5 rows



In [25]:
#c. utilizando funções
import pyspark.sql.functions as func
df.select('price','neighbourhood').where(func.col('price')>120).groupby('neighbourhood').count().withColumnRenamed('count', 'contagem').orderBy('contagem', ascending=False).show(5)


+------------------+--------+
|     neighbourhood|contagem|
+------------------+--------+
|         Manhattan|    1956|
|          Brooklyn|    1488|
|           Midtown|     319|
|      Williamsburg|     300|
|Bedford-Stuyvesant|     279|
+------------------+--------+
only showing top 5 rows



In [26]:
#4. selecionar apartamento mais caro por bairro
#a. sql puro
query = 'Select neighbourhood, max(price) as max_price from realstate group by neighbourhood order by max_price desc'
sc.sql(query).show(5)



+---------------+---------+
|  neighbourhood|max_price|
+---------------+---------+
|      Manhattan|     9999|
|       Brooklyn|     8000|
|Upper West Side|     3000|
|         Nolita|     2990|
| Hell's Kitchen|     2200|
+---------------+---------+
only showing top 5 rows



In [28]:
#b. selecionar apartamento mais caro por bairro utilizando dataframe Spark SQL
df.groupBy('neighbourhood').max('price').withColumnRenamed('max(price)', 'max_price').orderBy('max_price', ascending=False).show(5)



+---------------+---------+
|  neighbourhood|max_price|
+---------------+---------+
|      Manhattan|     9999|
|       Brooklyn|     8000|
|Upper West Side|     3000|
|         Nolita|     2990|
| Hell's Kitchen|     2200|
+---------------+---------+
only showing top 5 rows



In [29]:
#c. utilizando funções do Spark SQL
df.groupBy('neighbourhood').agg(func.max('price').alias('max_price')).orderBy('max_price', ascending=False).show(5)


+---------------+---------+
|  neighbourhood|max_price|
+---------------+---------+
|      Manhattan|     9999|
|       Brooklyn|     8000|
|Upper West Side|     3000|
|         Nolita|     2990|
| Hell's Kitchen|     2200|
+---------------+---------+
only showing top 5 rows



In [30]:
#5. Select do valor mais caro de Manhattan
#a. usando sql
query = 'Select max(price) from realstate where neighbourhood = "Manhattan"'
sc.sql(query).show()



+----------+
|max(price)|
+----------+
|      9999|
+----------+



In [31]:
#b. usando dataframe
df.select('price').where(df.neighbourhood == 'Manhattan').agg(func.max('price')).show()



+----------+
|max(price)|
+----------+
|      9999|
+----------+



In [32]:
#c. usando funções
df.select('price').where(func.col('neighbourhood')=='Manhattan').agg(func.max('price')).show()


+----------+
|max(price)|
+----------+
|      9999|
+----------+



In [60]:
###############################
## exercício - com SQL -- com dataframe e com funcoes
# #Converter o CSV (https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/F1_Race_Results.csv) para Spark SQL.
#add files
from pyspark.sql.functions import col
url = 'https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/F1_Race_Results.csv'
sc.sparkContext.addFile(url)
# criar dataframe
df = sc.read.csv(SparkFiles.get("F1_Race_Results.csv"), header=True, inferSchema=True)
#1. Mostrar em ordem decrescente os pilotos com maior número de vitórias.
df.groupBy('Winner').count().orderBy('count', ascending=False).show()
df=df.withColumn('Laps', col('Laps').cast('int'))
df.select('Winner','Laps').groupBy('Winner').avg('Laps').orderBy('avg(Laps)', ascending=False).show()
#2. Mostrar em ordem decrescente as equipes com maior número de vitórias.
df.groupBy('Car').count().orderBy('count', ascending=False).show()
#3. Mostrar em ordem decrescente as pistas com maior número de voltas acumuladas de toda história.
#4. Mostrar em ordem decrescente a relação de pilotos com mais vitórias por equipes.
#5. Mostrar em ordem decrescente a relação do número de voltas por pilotos a partir de corridas vencidas.

+--------------------+-----+
|              Winner|count|
+--------------------+-----+
|  Lewis Hamilton HAM|  103|
|Michael Schumache...|   91|
|Sebastian Vettel VET|   53|
|     Alain Prost PRO|   51|
|    Ayrton Senna SEN|   41|
| Fernando Alonso ALO|   32|
|   Nigel Mansell MAN|   31|
|  Jackie Stewart STE|   27|
|       Jim Clark CLA|   25|
|      Niki Lauda LAU|   25|
|Juan Manuel Fangi...|   24|
|   Nelson Piquet PIQ|   23|
|    Nico Rosberg ROS|   23|
|      Damon Hill HIL|   22|
|Kimi RÃ¤ikkÃ¶nen RAI|   21|
|   Mika Hakkinen HAK|   20|
|  Max Verstappen VER|   19|
|   Stirling Moss MOS|   16|
|   Jenson Button BUT|   15|
|    Jack Brabham BRA|   14|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----------------+
|              Winner|        avg(Laps)|
+--------------------+-----------------+
|     Lee Wallard WAL|            200.0|
|    Pat Flaherty FLA|            200.0|
|     Jimmy Bryan BRY|            200.0|
|     Rodger Ward WAR|          

In [35]:
url = 'https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/F1_Race_Results.csv'

#adicionar arquivo ao contexto
sc.sparkContext.addFile(url)

# carregando a base realestate ao Spark
df = sc.read.csv(SparkFiles.get("F1_Race_Results.csv"), header=True, inferSchema= True)
df.show(5)

df.createOrReplaceTempView('f1')

+----------------+-----------+--------------------+--------------------+----+-----------+
|       GrandPrix|       Date|              Winner|                 Car|Laps|       Time|
+----------------+-----------+--------------------+--------------------+----+-----------+
|   Great Britain|13 May 1950|     Nino Farina FAR|          Alfa Romeo|  70|2:13:23.600|
|          Monaco|21 May 1950|Juan Manuel Fangi...|          Alfa Romeo| 100|3:13:18.700|
|Indianapolis 500|30 May 1950|Johnnie  Parsons PAR|Kurtis Kraft Offe...| 138|2:46:55.970|
|     Switzerland|04 Jun 1950|     Nino Farina FAR|          Alfa Romeo|  42|2:02:53.700|
|         Belgium|18 Jun 1950|Juan Manuel Fangi...|          Alfa Romeo|  35|2:47:26.000|
+----------------+-----------+--------------------+--------------------+----+-----------+
only showing top 5 rows



In [ ]:
#1. Mostrar em ordem decrescente os pilotos com maior número de vitórias.
#a. sql


In [ ]:
#b.dataframe


In [ ]:
#c. funcoes


In [ ]:
#2. Mostrar em ordem decrescente as equipes com maior número de vitórias.
#a. sql


In [ ]:
#b.dataframe


In [ ]:
#c. funcoes


In [ ]:
#3. Mostrar em ordem decrescente as pistas com maior número de voltas acumuladas de toda história.
#a. sql


In [ ]:
#b.dataframe


In [ ]:
#c. funcoes


In [ ]:
#4. Mostrar em ordem decrescente a relação de pilotos com mais vitórias por equipes.
#a. sql


In [ ]:
#b.dataframe


In [ ]:
#c. funcoes


In [ ]:
#5. Mostrar em ordem decrescente a relação do número de voltas por pilotos a partir de corridas vencidas.
#a. sql


In [ ]:
#b.dataframe


In [ ]:
#c. funcoes


In [ ]:
#############################################
##    Pandas
#############################################

In [ ]:
##Pandas
#importar pandas
import pandas as pd

In [ ]:
#criar dataframe
url = 'https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/realstate.csv'

df = pd.read_csv(url, delimiter=',')

In [ ]:
# exibir


In [ ]:
#1. Select bairros e preços
#2. Select bairros e preços com acomodação com preço > 150
#3. Selecionar bairros e preços com preço > 150 e agrupado por bairro
#4. Selecionar apartamentos mais caros
#5. Selecionar apartamento mais caro  por bairro
#6. Selecionar o apartamento mais caro de Manhattan

In [ ]:
#1. Select bairros e preços


In [ ]:
#2. Select bairros e preços com acomodação com preço > 150


In [ ]:
#3. Fazer a contagem de apartamentos por bairros que possuem preço acima de 150


In [ ]:
#4. Selecionar apartamentos mais caros


In [ ]:
#5. Selecionar apartamento mais caro  por bairro


In [ ]:
#6. Selecionar o apartamento mais caro de Manhattan


In [ ]:
#7. Contar número de apartamentos por bairro


In [ ]:
####################################
## Atividades com Pandas
####################################

In [ ]:
#1. Mostrar em ordem decrescente os pilotos com maior número de vitórias.
url = 'https://raw.githubusercontent.com/joegotz/bigDataAnalytics/main/F1_Race_Results.csv'
df3 = pd.read_csv(url, delimiter=',')




In [ ]:
#2. Mostrar em ordem decrescente as equipes com maior número de vitórias.


In [ ]:
#3. Mostrar em ordem decrescente as pistas com maior número de voltas acumuladas de toda história.


In [ ]:
#4. Mostrar em ordem decrescente a relação de pilotos com mais vitórias por equipes.


In [ ]:
#5. Mostrar em ordem decrescente a relação do número de voltas por pilotos a partir de corridas vencidas.
